In [1]:
%load_ext autoreload
%autoreload 2

ndevice = 4
import os
import numpyro
numpyro.set_host_device_count(ndevice)
#import arviz as az
from astropy.cosmology import Planck18
import astropy.units as u
import sys
import jax
sys.path.append('../src/')
import intensity_models
import numpy as np
from numpyro.infer import MCMC, NUTS, SA
import os.path as op
import pandas as pd
import paths
from utils import get_priors_from_file
from intensity_models import coords
import jax.numpy as jnp
import tqdm
from scipy.special import logsumexp

/mnt/home/amoran/GW_pop_cosmo_inference/reproduce/../src/intensity_models.py:153: SyntaxWarning: invalid escape sequence '\m'
  :math:`m = m_\mathrm{BH,max}` with relative amplitude `fpl` and slope `c`.


In [2]:
jax.local_devices()

[CudaDevice(id=0)]

In [3]:
nmcmc = 2000
nchain = 1
random_seed = 1652819403

In [4]:
sel_samples = pd.read_hdf('../src/selection_samples_o3.h5', 'samples')
print(sel_samples.columns.tolist())


['m1', 'q', 'z', 'a1', 'a2', 'cos_tilt_1', 'cos_tilt_2', 'pdraw_m1sqz', 'ndraw', 'dm1sz_dm1ddl']


In [5]:
nsamp = 200  # number of posterior draws per event

pe_samples_mock = pd.read_hdf('../src/pe_samp_mocko3.hdf5', 'samples')
m1s = np.asarray(pe_samples_mock['m1'].to_list()) #detector frame already
qs = np.asarray(pe_samples_mock['q'].to_list())
dls = np.asarray(pe_samples_mock['dl'].to_list())
pdraws = np.asarray(pe_samples_mock['pdraw'].to_list())
print("array shapes (we want nevents, nsamples): ", m1s.shape, qs.shape, dls.shape, pdraws.shape)

array shapes (we want nevents, nsamples):  (583, 200) (583, 200) (583, 200) (583, 200)


In [6]:
prior = get_priors_from_file("priors/gwtc3_noevo.prior")
# https://dcc.ligo.org/public/0177/P2100239/009/o3_population.pdf?

sel_samples['pdraw_cosmo'] = sel_samples['pdraw_m1sqz'] * sel_samples['dm1sz_dm1ddl']
sel_samples['m1d'] = sel_samples['m1']*(1+sel_samples['z']) #source to detector frame
sel_samples['dl'] = Planck18.luminosity_distance(sel_samples['z'].to_numpy()).to(u.Gpc).value

In [7]:
ndraw = sel_samples['ndraw'].iloc[0]

kernel = NUTS(intensity_models.pop_cosmo_model)
mcmc = MCMC(kernel, num_warmup=nmcmc, num_samples=nmcmc, num_chains=nchain, progress_bar=True)
mcmc.run(jax.random.PRNGKey(random_seed),
         m1s, qs, dls, pdraws,
         sel_samples['m1d'], sel_samples['q'], sel_samples['dl'], sel_samples['pdraw_cosmo'], ndraw, prior)
samples = mcmc.get_samples(group_by_chain=True)  # shape: (nchain, nsamples, ...)
np.savez("mock_obs_1det_gc.npz", **samples)

sample: 100%|██████████| 4000/4000 [05:53<00:00, 11.33it/s, 7 steps of size 3.87e-01. acc. prob=0.91]   
